# HiHydroSoil v2.0 Data Processing

This repository contains a Jupyter Notebook script that processes raster data related to water holding capacity using ArcPy, a Python library for geospatial data analysis. The script extracts, processes, and projects raster data to generate water holding capacity maps for specific regions.

## Prerequisites

To run the script, ensure you have the following installed:

- ArcGIS Desktop with ArcPy (version compatible with the script)
- Required raster data files in the specified directories

## Usage

1. Open the Jupyter Notebook in your ArcGIS environment.
2. Modify the paths and parameters in the code to match your data and requirements.
3. Run the script cells sequentially to perform the data processing steps.

## Script Overview

The script performs the following main tasks:

### 1. Import Packages and Set Environment

The necessary Python packages are imported, and the ArcGIS environment is set up.

### 2. Water Holding Capacity Calculation

The script loads raster data representing water availability for different soil layers and calculates the water holding capacity for specific depth ranges. It also extracts and reprojects the Water content at field capacity and permanent wilting point over Switzerland Calculations are performed considering various soil layers' contributions.

### 3. Data Elaboration

The script processes data stored in specified folders by extracting and projecting raster data based on defined masks. The extracted data is then saved in new output folders, preserving geographical and spatial information.

## Important Notes

- Ensure that the ArcGIS environment is set up correctly before running the script.
- Modify input and output paths, mask paths, and other parameters as needed.
- Review and adjust the cell size, geographic transformation, and other parameters during raster projection.

## Author

Script written by Luca Ferrari
Contact: luca.ferrari@usys.ethz.ch

For any inquiries or assistance, please reach out to the author.

This README content was generated with the assistance of an AI language model from OpenAI. The provided content is based on user input and has been tailored to the specific requirements of the project.

In [ ]:
# %% Import packages
import arcpy
from arcpy import env
import os
import time
arcpy.CheckOutExtension("Spatial")

In [ ]:
# %% Define workspace
env.workspace = r"N:\Luca_data"
arcpy.env.overwriteOutput = True

# Specify the mask path
path_mask_Europe = r"\\Code\Extract_Europe_Data\EuropeMask\EuropeMask.shp"
path_mask_Switzerland = r"\\Code\Extract_Europe_Data\SwissMask\SwissMask.shp"


In [ ]:
# Water Holding Capacity Global 15 cm
# Load the raster data of water holding capacity
WHC_0_5 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail\WCavail_0-5cm_M_250m.tif'
WHC_0_5 = arcpy.Raster(WHC_0_5)

WHC_5_15 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail\WCavail_5-15cm_M_250m.tif'
WHC_5_15 = arcpy.Raster(WHC_5_15)

# evaluate the volumetric water content for a single soil layer ( multiplied by 0.1 because it has to be multiplied by 0.0001 to rescale it  to m3/m3 and by 1000 to convert to mm/m)
WHC_tot = (WHC_0_5*0.05 + WHC_5_15*0.10) * 0.1 

WHC_tot.save(r'N:\Luca_data\HiHydroSoil v2.0\WHC\WCavail_0-15cm_M_250m.tif')

In [ ]:
# Water Holding Capacity Global 60 cm
# Load the raster data of water holding capacity
WHC_0_5 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail\WCavail_0-5cm_M_250m.tif'
WHC_0_5 = arcpy.Raster(WHC_0_5)

WHC_5_15 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail\WCavail_5-15cm_M_250m.tif'
WHC_5_15 = arcpy.Raster(WHC_5_15)

WHC_15_30 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail\WCavail_15-30cm_M_250m.tif'
WHC_15_30 = arcpy.Raster(WHC_15_30)

WHC_30_60 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail\WCavail_30-60cm_M_250m.tif'
WHC_30_60 = arcpy.Raster(WHC_30_60)

# evaluate the volumetric water content for a single soil layer
WHC_tot = (WHC_0_5*0.05 + WHC_5_15*0.10 + WHC_15_30*0.15 + WHC_30_60*0.30) * 0.0001 * 1000

WHC_tot.save(r'N:\Luca_data\HiHydroSoil v2.0\WHC\WCavail_0-60cm_M_250m.tif')



In [ ]:
folders = ["WCavail"]

# %% Elaborate data
for folder in folders:
    # Specify the input path
    input_path = os.path.join(env.workspace, "HiHydroSoil v2.0", folder)

    # Specify the output path
    output_path = os.path.join(env.workspace, "HiHydroSoil v2.0", f"{folder}_Switzerland")
    
    # Create output folder if it does not exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Get the list of files in the directory using os.scandir()
    with os.scandir(input_path) as entries:
        # Filter out directories and get only file names
        file_names = [entry.name for entry in entries if entry.is_file()]
        # Filter the list to only include .tif files
        file_names = [f for f in file_names if f.endswith('.tif')]

    # Process data for each file name and output file path
    for file_name in file_names:

        # Construct full paths to the file and output
        file_path = os.path.join(input_path, file_name)
        file_name_without_extension = os.path.splitext(file_name)[0]
        new_file_name = f"{file_name_without_extension}_Switzerland.tif"
        output_file_path = os.path.join(output_path, new_file_name)
        
        # Perform processing using ExtractByMask function
        extract_raster = arcpy.sa.ExtractByMask(
            in_raster=file_path,
            in_mask_data=path_mask_Switzerland,
            extraction_area="INSIDE",
            analysis_extent='5.95606700000008 45.81836186 10.4920640000001 47.807378701 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
        )
 
        # Project the output raster
        arcpy.management.ProjectRaster(
            in_raster=extract_raster,
            out_raster=output_file_path,
            out_coor_system='PROJCS["CH1903+_LV95",GEOGCS["GCS_CH1903+",DATUM["D_CH1903+",SPHEROID["Bessel_1841",6377397.155,299.1528128]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Hotine_Oblique_Mercator_Azimuth_Center"],PARAMETER["False_Easting",2600000.0],PARAMETER["False_Northing",1200000.0],PARAMETER["Scale_Factor",1.0],PARAMETER["Azimuth",90.0],PARAMETER["Longitude_Of_Center",7.439583333333333],PARAMETER["Latitude_Of_Center",46.95240555555556],UNIT["Meter",1.0]]',
            resampling_type="NEAREST",
            cell_size="213.670263063039 213.670263063039",
            geographic_transform="CH1903+_To_WGS_1984_1",
            Registration_Point=None,
            in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
            vertical="NO_VERTICAL"
        )
        
        print(f"Processed: {file_path} -> {output_file_path}\n")


In [ ]:
# Load the raster data of water holding capacity
WHC_0_5 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail_Switzerland\WCavail_0-5cm_M_250m_Switzerland.tif'
WHC_0_5 = arcpy.Raster(WHC_0_5)

WHC_5_15 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail_Switzerland\WCavail_5-15cm_M_250m_Switzerland.tif'
WHC_5_15 = arcpy.Raster(WHC_5_15)

WHC_15_30 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail_Switzerland\WCavail_15-30cm_M_250m_Switzerland.tif'
WHC_15_30 = arcpy.Raster(WHC_15_30)

WHC_30_60 = r'N:\Luca_data\HiHydroSoil v2.0\WCavail_Switzerland\WCavail_30-60cm_M_250m_Switzerland.tif'
WHC_30_60 = arcpy.Raster(WHC_30_60)

# evaluate the volumetric water content for a single soil layer
WHC_tot = (WHC_0_5*0.05 + WHC_5_15*0.10 + WHC_15_30*0.15 + WHC_30_60*0.30) * 0.0001 * 1000

WHC_tot.save(r'N:\Luca_data\HiHydroSoil v2.0\WHC\WCavail_0-60cm_M_250m_Switzerland.tif')

In [ ]:
# %% Elaborate data

folders = ["WCpF2", "WCpF4.2"]

for folder in folders:
    # Specify the input path
    input_path = os.path.join(env.workspace, "HiHydroSoil v2.0", folder)

    # Specify the output path
    output_path = os.path.join(env.workspace, "HiHydroSoil v2.0", f"{folder}_Switzerland")
    
    # Create output folder if it does not exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Get the list of files in the directory using os.scandir()
    with os.scandir(input_path) as entries:
        # Filter out directories and get only file names
        file_names = [entry.name for entry in entries if entry.is_file()]
        # Filter the list to only include .tif files
        file_names = [f for f in file_names if f.endswith('.tif')]

    # Process data for each file name and output file path
    for file_name in file_names:

        # Construct full paths to the file and output
        file_path = os.path.join(input_path, file_name)
        file_name_without_extension = os.path.splitext(file_name)[0]
        new_file_name = f"{file_name_without_extension}_Switzerland.tif"
        output_file_path = os.path.join(output_path, new_file_name)
        
        # Perform processing using ExtractByMask function
        extract_raster = arcpy.sa.ExtractByMask(
            in_raster=file_path,
            in_mask_data=path_mask_Switzerland,
            extraction_area="INSIDE",
            analysis_extent='5.95606700000008 45.81836186 10.4920640000001 47.807378701 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
        )
 
        # Project the output raster
        arcpy.management.ProjectRaster(
            in_raster=extract_raster,
            out_raster=output_file_path,
            out_coor_system='PROJCS["CH1903+_LV95",GEOGCS["GCS_CH1903+",DATUM["D_CH1903+",SPHEROID["Bessel_1841",6377397.155,299.1528128]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Hotine_Oblique_Mercator_Azimuth_Center"],PARAMETER["False_Easting",2600000.0],PARAMETER["False_Northing",1200000.0],PARAMETER["Scale_Factor",1.0],PARAMETER["Azimuth",90.0],PARAMETER["Longitude_Of_Center",7.439583333PWP3333],PARAMETER["Latitude_Of_Center",46.95240555555556],UNIT["Meter",1.0]]',
            resampling_type="NEAREST",
            cell_size="213.670263063039 213.670263063039",
            geographic_transform="CH1903+_To_WGS_1984_1",
            Registration_Point=None,
            in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
            vertical="NO_VERTICAL"
        )
        
        print(f"Processed: {file_path} -> {output_file_path}\n")
